In [16]:
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
import time
from utils.general import *
import numpy as np
import pickle


In [17]:
class MySentenceCollection:
    def __init__(self, filename):
        self.filename = filename
        self.rd = None

    def __iter__(self):
        self.rd = open(self.filename, 'r', encoding='utf-8', newline='\r\n')
        return self

    def __next__(self):
        line = self.rd.readline()
        if line:
            return list(line.strip('\r\n').split(' '))
        else:
            self.rd.close()
            raise StopIteration


In [20]:
InFile_dir = 'data_folder/my'
OutFile_dir = 'data_folder/my/pretrained-embeddings'
OutFile_dir_KG = 'data_folder/my/KG'
OutFile_dir_DKN = 'data_folder/my/DKN-training-folder'

In [21]:
def train_word2vec(Path_sentences, OutFile_dir):     
    OutFile_word2vec = os.path.join(OutFile_dir, r'word2vec.model')
    OutFile_word2vec_txt = os.path.join(OutFile_dir, r'word2vec.txt')

    create_dir(OutFile_dir)

    my_sentences = MySentenceCollection(Path_sentences)

    model = Word2Vec(my_sentences, size=32, window=5, min_count=1, workers=4, iter=50)

    model.save(OutFile_word2vec)
    model.wv.save_word2vec_format(OutFile_word2vec_txt, binary=False)

Path_sentences = os.path.join(InFile_dir, 'sentence.txt')
train_word2vec(Path_sentences, OutFile_dir)

KeyboardInterrupt: 

In [26]:
## some step in transE training

## https://github.com/thunlp/Fast-TransX

!bash ./run_transE.sh

/data/home/jialia/jialia/kdd2020tutorial/formal/recommenders/scenarios/KDD2020-tutorial
fatal: destination path 'Fast-TransX' already exists and is not an empty directory.
./run_transE.sh: line 14: ./transE2: No such file or directory


In [30]:

def gen_context_embedding(entity_file, context_file, kg_file):
    #load embedding_vec
    entity_index = 0
    entity_dict = {}
    fp_entity = open(entity_file, 'r')
    for line in fp_entity:
        linesplit = line.strip().split('\t')[:EMBEDDING_LENGTH]
        linesplit = list(map(float, linesplit))
        entity_dict[str(entity_index)] = linesplit
        entity_index += 1
    fp_entity.close()

    #build neighbor for entity in entity_dict
    fp_kg = open(kg_file, 'r', encoding='utf-8')
    triple_num = fp_kg.readline()
    triples = fp_kg.readlines()
    kg_neighbor_dict = {}
    for triple in triples:
        linesplit = triple.strip().split(' ')
        head = linesplit[0]
        tail = linesplit[1]
        if head not in kg_neighbor_dict:
            kg_neighbor_dict[head] = set()
        kg_neighbor_dict[head].add(tail)

        if tail not in kg_neighbor_dict:
            kg_neighbor_dict[tail] = set()
        kg_neighbor_dict[tail].add(head)        
    fp_kg.close()

    context_embeddings = np.zeros([entity_index , EMBEDDING_LENGTH])

    for entity in entity_dict:
        if entity in kg_neighbor_dict:
            context_entity = kg_neighbor_dict[entity]
            context_vecs = []
            for c_entity in context_entity:
                context_vecs.append(entity_dict[c_entity])

            context_vec = np.mean(np.asarray(context_vecs), axis=0)
            context_embeddings[int(entity)] = context_vec

    np.savetxt(context_file, context_embeddings, delimiter='\t')

In [32]:
##### build context embedding
EMBEDDING_LENGTH = 32
entity_file = os.path.join(OutFile_dir_KG, 'entity2vec.vec') 
context_file = os.path.join(OutFile_dir_KG, 'context2vec.vec')   
kg_file = os.path.join(OutFile_dir_KG, 'train2id.txt')   
gen_context_embedding(entity_file, context_file, kg_file)

In [33]:
load_np_from_txt(
        os.path.join(OutFile_dir_KG, 'entity2vec.vec'),
        os.path.join(OutFile_dir_DKN, 'entity_embedding.npy'),
    )
load_np_from_txt(
        os.path.join(OutFile_dir_KG, 'context2vec.vec'),
        os.path.join(OutFile_dir_DKN, 'context_embedding.npy'),
    )
format_word_embeddings(
    os.path.join(OutFile_dir, 'word2vec.txt'),
    os.path.join(InFile_dir, 'word2idx.pkl'),
    os.path.join(OutFile_dir_DKN, 'word_embedding.npy')
)
